<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/PIL_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
import time
def sleepyTime():
    count=0
    try:
        while count <= 120:
            print(f'sleeping for {count} minutes')
            time.sleep(60)
            count+=1
    except KeyboardInterrupt:
        clear_output()
# sleepyTime()
print('ok')

In [ ]:
from __future__ import absolute_import
try:
    import tensorflow as tf
    print(f'tensorflow: {tf.__version__}')
except ModuleNotFoundError as err:
    %pip install tensorflow
    import tensorflow as tf
    print(f'\ntensorflow: {tf.__version__}')

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from IPython.display import Image, display, clear_output
from tensorflow.keras.applications import imagenet_utils
import numpy as np

import glob, os, shutil, tarfile
from os.path import *
import numpy as np

try: import PIL
except ModuleNotFoundError:
    %pip install Pillow
    import PIL
    
try: import matplotlib.image as mpimg
except ModuleNotFoundError:
    %pip install matplotlib
    import matplotlib.image as mpimg
    
try:
    from google.colab import drive, files
    drive.flush_and_unmount()
    drive.mount('/content/gdrive', force_remount=True)
except: pass

contentPth = os.getcwd()
gdrivePth = join(contentPth, 'gdrive')
myDrivePth = join(gdrivePth, 'My Drive')
modelPth = join(myDrivePth, 'TensorflowModels')
imgPth = join(myDrivePth, 'TensorflowImages')
imgDirectoryPth = join(contentPth, 'Defcon_Images')

deletePth = join(contentPth, 'sample_data')
if exists(deletePth):
    shutil.rmtree(deletePth)
    deletePth=None

try:
    os.chdir(myDrivePth)
    from BashColors import C
except FileNotFoundError as err:
    # print(err)
    from BashColors import C
    
imagePathList=[]
print(f'cwd: {C.Blue}{os.getcwd()}')

In [ ]:

tf.debugging.set_log_device_placement(True)
print(tf.config.list_physical_devices())
with tf.device('GPU'):
    print('ok')
tf.device('GPU')

In [ ]:
import matplotlib.pyplot as plt

from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
def resize_rescale(inputImage, imageSize=224):
    resize_and_rescale = tf.keras.Sequential([Resizing(imageSize, imageSize, 3),
                                              Rescaling(1.0/255)])
    result = resize_and_rescale(inputImage)
    return result


In [ ]:
def rotateImages(nextImage, rotationAngle):
    imageCount=0
    print(nextImage)
    from PIL import Image as pilImage
    try:
        img = pilImage.open(nextImage) # Relative path
        img.save(nextImage)
        # showImage(nextImage, newWidth=224, newHeight=224)
        
        # angleList=[45, 90, 135, 180, 225, 270, 315]
        angleList=[rotationAngle]
        for angle in angleList:
            imageCount+=1
            img = img.rotate(angle)
            newName = nextImage + '_rotated_' + str(angle) + '.png'
            # print(f'{C.BBlue}{newName}{C.ColorOff}')
            img.save(newName)
            fullPth = abspath(newName)
            # print(fullPth)
            showImage(fullPth, newWidth=224, newHeight=224)
    except IOError as err:
        print(err)
print('ok')

In [ ]:
result=None
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
def showImage(thisImagePath, newWidth=64, newHeight=64):
    if exists(thisImagePath):
        image_name=basename(thisImagePath)
        print(f'{C.Green}{thisImagePath}')
        print(f'{C.IWhite}{image_name}')

        thisImage = image.load_img(path=thisImagePath, 
                                       target_size=(newWidth, 
                                                    newHeight))
        imgArray = image.img_to_array(img=thisImage)
        imgArrayExpandDims = np.expand_dims(a=imgArray, axis=0)
        result=preprocess_input(imgArrayExpandDims)
        # result = tf.keras.applications.mobilenet_v2.preprocess_input(imgArrayExpandDims)
        show = Image(filename=thisImagePath, width=newWidth,height=newHeight)
        display(show)

print('ok')
print('Image: ', Image)

In [ ]:
try:
    tar = tarfile.open('originalDefconImages.tar.gz', 'r:gz')
    tar.extractall()
    tar.close()
except FileNotFoundError:
    if exists(myDrivePth):
        os.chdir(myDrivePth)
    else: os.chdir(contentPth)
    print(f'cwd: {C.Blue}{os.getcwd()}')
    tar = tarfile.open('Original_Defcon_Images.tar.gz')
    tar.extractall()
    tar.close()

In [ ]:
fileList=glob.glob('**', recursive=True)
for fil in sorted(fileList):
    if isfile(fil) and fil.endswith('.png'):
        print(f'{C.BPurple}{fil}')
        newFilePth=join(contentPth, basename(fil))
        if not exists(newFilePth):
            shutil.copy2(fil, contentPth)
            print(f'{C.Green}{newFilePth}\n')
        elif exists(newFilePth):
            print(f'{C.Green}{newFilePth}\n')

In [ ]:
os.chdir(contentPth)

imageListGlob=[]
print(f'imageListGlob: {len(imageListGlob)} images')
imageCount=0
 
for img in sorted(imageListGlob):
    if img.__contains__('rotated'):
        imageListGlob.remove(img)
        pass
    else:
        fullPath=abspath(img)
        imageCount+=1
        # renameImage(thisImg=img, count=imageCount)
        showImage(fullPath)

In [ ]:
imageListGlob=[]
imageNameList=[]
os.chdir(contentPth)
imageListGlob=glob.glob('**', recursive=False)
for img in sorted(imageListGlob):
    if img.endswith('.png') and not img.__contains__('rotated'):
        fullPth = abspath(img)
        print(img)
        imageNameList.append(img)
        # print(fullPth)
        # shutil.copy2(fullPth, contentPth)

In [ ]:
for name in sorted(imageNameList):
    # print(name)
    fullName=abspath(name)

    #showImage(thisImagePath=fullName,
              #newWidth=512, newHeight=512)
    
    rotateImages(nextImage=name, rotationAngle=90)
    rotateImages(nextImage=name, rotationAngle=180)
